# Le modèle de ségrégation de Schelling

En Décembre 2016 disparaissait Thomas C. Schelling. Economiste, récipiendaire du Prix de la Banque de Suède en sciences économiques en mémoire d’Alfred Nobel (communément appelé –et considéré– comme le prix Nobel d’Economie), il a travaillé sur de nombreux sujets, en particulier l’analyse des conflits.

Une de ses contributions a permis de mieux comprendre les phénomènes de ségrégation. Plus précisément, le propos de Schelling fut d’étu- dier la dynamique par laquelle des phénomènes de ségrégation extrêmes peuvent survenir, en dépit de préférences qui peuvent sembler faiblement discriminantes individuellement. Ainsi, même si chaque individu se dé- clare prêt à accepter une certaine proportion d’invidus « différents » dans son voisinage, le résultat final peut être que la population se regroupe en régions très homogènes.
Cette étude peut être menée à l’aide de modèles connus sous le nom d’automates cellulaires. Un des automates les plus célèbres est le jeu de la vie, proposé par John Conway en 1970. Il existe de nombreux autres automates cellulaires dont le comportement est relativement bien étudié, et la littérature est riche à ce sujet. Nous recommandons la lecture de l’article [2] de Jean-Paul Delahaye.

In [84]:
import numpy as np
import random
import copy

from matplotlib import pyplot as plt

# GLOBAL PARAMETERS FOR EXPERIMENTS
neighb = 4          # size of neighbourhood
threshold = 0.5     # threshold of satisfaction
maxIterations = 5   # max number of iteration for convergence

Un état de l'automate cellulaire est représenté sous la forme d'une liste de 0 et de 1. Par exemple:

In [96]:
[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]

[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]

La fonction ```str_state``` convertit un automate cellulaire en une chaîne de caractères pour l'afficher à l'écran.

In [66]:
def str_state(state):
    '''
    return the state as a string
    '''
    result = ""
    for i in state:
        result += str(i)
    return result

In [67]:
str_state([0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,0,1,1,1,1,0,1])

'0100011010011100111101'

La fonction ```hogeneinity_level``` calcule pour l'individu à la position c, le ratio d'individus du même type.

In [82]:
def homogeneinity_level(position, state):
    '''
    for a given individual at position and state
    returns the ratio of individuals of same type in neighbourhood
    '''
    my_color = state[position]
    count = 0
    nb_neighb = 0
    for i in range(1, neighb+1):
        if position+i < len(state):
            nb_neighb += 1
            if state[position+i] == my_color:
                count += 1
        if position-i >= 0:
            nb_neighb += 1
            if state[position-i] == my_color:
                count += 1
    return float(count / nb_neighb)

In [83]:
homogeneinity_level(1,[0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,1,1,1,0,1])

0.2

In [ ]:
def is_happy(position, state, verbose = False):
    '''
    returns whether individual at position is satisfied in a given state
    '''
    h = homogeneinity_level(position, state)
    if verbose:
        print(h)
    return h >= threshold

In [94]:
is_happy(1, [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1])

False

In [ ]:
def str_unhappy(string):
    '''
    returns the string marking unhappy individuals with a 'X'
    '''
    result = ""
    for i in range(len(string)):
        if is_happy(i, string):
            result += " "
        else:
            result += "X"
    return result

In [89]:
str_unhappy(str_state([0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,1,1,1,0,1]))

' X   XX  X X XX    X '

La fonction ```move_to``` déplace un individu vers une nouvelle position à droite ou à gauche. La priorité est donnè au déplacement à droite.

In [106]:
def move_to(c, p, s):
    '''
    c position initiale
    p position où l'on déplace l'agent
    moves individual c to position p, shifting other individuals
    s state
    and returns the resulting list
    '''
    new_s = copy.copy(s) # new list for result
    my_color = new_s[c]
    # gives priority to the right move in case of ties
    # I didn't find this specification in Schelling's paper
    if p > c: # moving to the right
        print(p)
        print(c)
        for i in range(c, p):
            new_s[i] = new_s[i+1]
            new_s[i+1] = my_color
    else:   # moving to the left

        for i in range(p, c):
            new_s[i+1] = new_s[i]
            new_s[p] = my_color
    return new_s

In [107]:
move_to(1, 7, [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1])

7
1


[0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]

In [108]:
def move_to_nearest_satisfying(c, s, verbose=False):
    '''
    will move individual c to nearest satisfying location
    simulate the move and check whether satisfying
    note: very inefficient but simple solution
    '''

    move_limit = max(size-c, c)
    move_distance = 0
    new_s = []
    satisfied = False
    while move_distance < move_limit and not(satisfied):
        move_distance += 1
        new_s = copy.copy(s) # used to simulate the move
        if c+move_distance < size:
            new_s = move_to(c, c+move_distance, new_s)
            if is_happy(c+move_distance, new_s):
                satisfied = True
                if verbose:
                    print (c, "moved to:", c+move_distance)
        else: # trying to move left
            if c-move_distance >= 0:
                new_s = copy.copy(s)
                new_s = move_to(c, c-move_distance, new_s)
                satisfied = is_happy(c-move_distance, new_s)
                if verbose and satisfied:
                    print (c, " moved to:", c-move_distance)
    return new_s, satisfied

In [109]:
move_to_nearest_satisfying(1, [0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,1,1,1,0,1])

2
1
3
1
4
1
5
1
6
1
7
1


([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1], True)

In [110]:
move_to_nearest_satisfying(1, [0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,1,1,1,0,1], verbose = True)

2
1
3
1
4
1
5
1
6
1
7
1
1 moved to: 7


([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1], True)

In [77]:
###############################################################################
# GLOBAL DYNAMICS
###############################################################################
# Note: departs a little bit from Schelling's specification here
# Interesting problem of non convergence here when no maxIterations condition
# the penultimate individual moves to the last position, and so on


def dynamics(state, verbose = False, stepwise = False):
    '''
    departs a little bit from Schelling's specification here
    '''
    moved = True
    iterations = 0
    while moved and iterations < maxIterations:
        moved = False
        for i in range(len(state)):
            if not (is_happy(i, state)):
                s, moved = move_to_nearest_satisfying(i, state, verbose=True)
        if verbose:
            print(str_state(state))
            print(str_unhappy(state))
            print(count_unhappy(state))
        if stepwise:
            input("Press Enter to continue...")
        iterations += 1
    return state

In [111]:
dynamics([0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1])

2
1
3
1
4
1
5
1
6
1
7
1
1 moved to: 7
6
5
7
5
8
5
5 moved to: 8
7
6
8
6
6 moved to: 8
10
9
9 moved to: 10
12
11
11 moved to: 12
14
13
15
13
16
13
17
13
18
13
19
13
20
13
21
13


IndexError: list index out of range

In [92]:
###############################################################################
# METRICS: A FAIRE
# count_unhappy(), average_homogeneity(), average_cluster_size()
###############################################################################


###############################################################################
#  SIMULATIONS: A FAIRE
###############################################################################


###############################################################################
# TESTING
###############################################################################


# Schelling original list

cells = [0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,1,1,0,1,1,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1,1,0,1,0,1,1,0]

sample_size = 100

# Printing the list and some metrics

print(str_state(cells))
print(str_unhappy(cells))
print("number of 0/1 unsatisfied:", count_unhappy(cells))
print("average level of satisfaction:", average_homogeneity(cells))
print("average cluster size:", average_cluster_size(cells))

# Testing moving agent 1 in the initial Schelling list

new_cells,_ = move_to_nearest_satisfying(1, cells, True)
print(str_state(new_cells))

# Testing simulations, for neighbourhood from 1 to 8

rslt = simulations([1, 2, 3, 4, 5, 6, 7, 8], 0.5, sample_size)
draw_simulations(rslt, sample_size)

0100011010011001110110110011001100110101001110110000011100010011010110
 X   XX  X X XX   X  X  X        X   XX XX   X X     XXX   X  X X X XX


NameError: name 'count_unhappy' is not defined